# Week 11


## This notebook requires that you have Anaconda installed. Please reach out to me or the TAs if you have questions regarding the installation of Anaconda!

Hello class! This week we'll finally be doing some analytics. Specifically, we'll be looking at the housing price data set, which you should download from Google Drive. The file "data_description.txt" contains the definition of each colunmn in the data set. We'll introduce the process of reading in data (through Pandas), creating visualization (through matplotlib), and creating a regression model to make predictions (through sklearn). These are essential packages for data scientists using python.

This notebook file is meant to be self-contained -- try executing the cells in this notebook one by one and make sure you understand what's going on in each cell and how different components make up the entire data analytics process.

In [ ]:
### Data source: https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data
### Adopted from https://www.dataquest.io/blog/kaggle-getting-started/


### Step 1: Importing useful packages and loading data

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv('train.csv')



### Step 2: Get a basic understanding of the dataframes



In [ ]:
print("Data shape:", data.shape) #data.shape shows (number of rows, number of columns) stored in the dataframe "data".


In [ ]:
data.head() # head() show the first few rows in the dataframe


### Import the matplotlib package to do visualization

In [ ]:
%matplotlib inline
# this ensures the plots are displayed inside the notebook
import matplotlib.pyplot as plt


#### Say we are interested in the SalePrice column in the 'data' dataframe. 
We can use describe() to check the summary statistics of this column. We can also create a simple histogram to get a sense of the distribution of values in this column.

In [ ]:
data.SalePrice.describe()


In [ ]:
# compute the skewness measure of the column
print("Skew is:", data.SalePrice.skew())

# create a histogram  in blue
plt.hist(data.SalePrice, color='blue')

# display the histogram
plt.show()

As shown in the plot above, the SalePrice column seems a bit skewed and doesn't have a nice bell shape.
We can easily transform the column to make the column more like a bell shape. This can often improve the prediction performance. For example, below we use the log transformation on SalePrice. We can see the skewness measure is a lot smaller, suggesting the column is not as skewed anymore.

In [ ]:
target = np.log(data.SalePrice)
print ("Skew is:", target.skew())
plt.hist(target, color='green')
plt.show()

### Select a subset of columns based on data type

Pandas DataFrame provides a handy function to help us filter columns we want. For example, suppose we only want to analyze columns containing numerical values (e.g., int, float, etc.). We can call the "select_dtypes" function on a DataFrame object and provide the argument "include=[np.number]" to tell pandas we only want numerical columns.

To get more help on the select_dtypes function, type and execute the following line in a separate cell

**help(pd.DataFrame.select_dtypes)**

In [ ]:
# generate a new DataFrame called numeric_features that only contains numerical columns
numeric_features = data.select_dtypes(include=[np.number])

# display the data type of each column in this new DataFrame
numeric_features.dtypes ## everything is either int or float!

### Correlation Matrix
You can use the corr() function to return the correlation for each pair of columns.
Correlation is between -1 and 1. Specifically, 1 means the pair of columns are perfectly positively correlated; -1 means the pair of columns are perfectly negatively correlated.

In [ ]:
numeric_features.corr()

In [ ]:
# save the correlation matrix to a DataFrame called corr
corr = numeric_features.corr()
print(corr['SalePrice'].sort_values(ascending=False)[:5], '\n')
print(corr['SalePrice'].sort_values(ascending=False)[-5:])

In [ ]:
# Since corr is a DataFrame, we can type corr['SalePrice'] to get the correlation between SalePrice and any other column
print(corr['SalePrice'])

In [ ]:
# We can choose to print the top 5 columns that have the highest correlation with SalePrice
print(corr['SalePrice'].sort_values(ascending=False)[:5])

In [ ]:
# We can also choose to print the top 5 columns that have the lowest correlation with SalePrice
print(corr['SalePrice'].sort_values(ascending=False)[-5:])

### One way to visualize the relationship between two numerical columns is to create a scatterplot:

In [ ]:
# Create a scatterplot where the x-axis is GrLivArea, and y-axis is target (log-transformed SalePrice) 
plt.scatter(x=data['GrLivArea'], y=target, color = 'orange')

# Add a label to the y-axis
plt.ylabel('Sale Price')

# Add a label to the x-axis
plt.xlabel('Above grade (ground) living area square feet')

# Display the plot: we can see the price increases as the living area (sqft) increases
plt.show()

In [ ]:
# We can do the same thing to understand the relationship between GarageArea and log(SalePrice)
plt.scatter(x=data['GarageArea'], y=target)
plt.ylabel('Sale Price')
plt.xlabel('Garage Area')
plt.show()

As we saw last week, we can easily filter our data based on certain criterion. **Such filtering is often used to get rid of outliers in the data set.** For example, as shown in the previous plot, there are several houses with huge garages but low sale prices that don't seem to be consistent with the general trend. In such cases, it's often useful to remove these outliers so our model can capture the general trend in the data rather than being affected by a small number of outliers.

Below, we only keep observations where the GarageArea < 1200.

In [ ]:
data = data.loc[data['GarageArea'] < 1200]

In [ ]:
plt.scatter(x=data['GarageArea'], y=np.log(data.SalePrice))
plt.xlim(-200,1600) # This forces the same scale as before
plt.ylabel('Sale Price')
plt.xlabel('Garage Area')
plt.show()

We can use the select_dtypes() function again, this time only returning the columns with non-numercial values


In [ ]:
categoricals = data.select_dtypes(exclude=[np.number])



Then, we can analyze these non-numerical columns using the describe() function. As these are now non-numerical columns, the summary statistics returned will be 'count', 'unique' (the number of unique values), 'top' (the most frequent value), 'freq' (the number of times the most frequent value appears).

In [ ]:
categoricals.describe()

In [ ]:
## Here we investigate a specific column (Street)


print ("Original: \n")
print (data.Street.value_counts(), "\n") # This shows the frequency of each value in this column.

## Step 3: Data Processing and Feature Generation

### Apply a function to each observation
To apply a certain function to each observation (row) in the DataFrame, we can define the function (e.g., myfunc()) we want to use, and then call **apply(myfunc)**.

For example, below we first analyze the *SaleCondition* column:

In [ ]:
data.SaleCondition.value_counts()

We see that there are 6 different values (e.g., Normal, Partial, Abnorml, ...) in this column. Say I only care whether or not a given house's sale condition is Normal (as opposed to forecloruse, family sales, etc.). In other words, I want to create a new column called **Is_Normal**, where Is_Normal = 1 if SaleCondition == 'Normal', and Is_Normal = 0 if SaleCondition is not 'Normal'.

What I can do is to create a simple function called check_normal() to return 1 or 0 depending on the value of some variable x. Then I apply this function to the SaleCondition column in the DataFrame by using the following syntax:

**data['Is_Normal'] = data.SaleCondition.apply(check_normal)**

Notice that in the above line I also made sure to save the returned value (1 or 0) to a new column called 'Is_Normal'.

In [ ]:
def check_normal(x):
    return 1 if x == 'Normal' else 0
data['Is_Normal'] = data.SaleCondition.apply(check_normal)

I can then verify that the 'Is_Normal' column has only two values:

In [ ]:
data.Is_Normal.value_counts()

We can create a bar chart to visualize the SalePrice difference between normal houses and abnormal houses. Below we plot the median for each group.

In [ ]:
condition_pivot = data.pivot_table(index='Is_Normal', values='SalePrice', aggfunc=np.median)
condition_pivot.plot(kind='bar', color='brown')
plt.xlabel('Normal Sale Condition (1) or else (0)')
plt.ylabel('Median Sale Price')
plt.xticks(rotation=0)
plt.show()

### Checking the "missingness" of the data set:
One annoying issue we need to address is the presence of missing values (i.e., Null values) in the data. In a real world data set, it is highly likely that not all observations have complete information, and thus we will often see missing values. Below we create a DataFrame to show how many missing values each column contains:

In [ ]:
nulls = pd.DataFrame(data.isnull().sum().sort_values(ascending=False)[:25])
nulls.columns = ['Null Count']
nulls.index.name = 'Feature'
nulls

#### Interpolate Missing Values
Many models/algorithms require that an observation contain complete information (i.e., no missing value). Any rows with missing values cannot be used by such models and will be dropped. When your data set is not large it's already challenging to make good predictions. In this case you really want to make sure to use all observations you have even though some observations might be missing a variable value here and there.

One way to still use such incomplete observations is to **interpolate** missing values. Essentially this is using certain models to *predict* what those missing values should have been, and fill those values in the blanks.

Below we show it's easy to interpolate: DataFrame objects have a function called *interpolate()* that does interpolation for you. type *help(pd.DataFrame.interpolate)* to check more interpolate options.

In [ ]:
#help(pd.DataFrame.interpolate)

In [ ]:
# Call the interpolate function to interpolate missing values.
# Then save the complete (with missing values filled) data back to the original DataFrame

data = data.select_dtypes(include=[np.number]).interpolate()


After interpolation, we can verify that there is no longer any missing values in the DataFrame:

In [ ]:
data.isnull().sum().sort_values(ascending=False)

After filling in missing values we are finally ready to make predictions.

# Step 4: Use linear regression to predict a continuous value

One thing we are interested in is whether we can predict a house's sale price using available information in the data set. A simple yet powerful machine learning model we can use is **linear regression**. We will train a linear regression model to let the model make predictions for us.

When we say "train an model" we really mean that we'll let the model see many different houses. For each house the model will study the house's characteristics and the corresponding sale price. The idea is that after seeing enough houses and their sale prices, the model will be able to *understand* or *learn* the underlying pattern/relationship between SalePrice and other house characteristics on its own. We can then utilize this learned pattern to make predictions for houses that haven't been sold. This is the basic idea behind any machine learning technique. 

To train a linear regression model to predict the SalePrice, we need to explicitly tell the model that SalePrice is the variable we want to predict (a.k.a., outcome variable, Y variable). We also want to specify the set of variables the model can use to make predictions (a.k.a., predictors, X variables). In our case, the predictors will just be all other variables in the data set except for the house ID, because we don't expect an arbitrary ID number will be predictive of the Sale Price.

In [ ]:
y = data.SalePrice # predict the SalePrice
X = data.drop(['SalePrice', 'Id'], axis=1)

## Small but important digression: The importance of splitting data into a training set and a test set (make sure to do this in your project!)
In a typical prediction task, the goal is to train a model that can make accurate predictions of an outcome variable of interest e.g., SalePrice). Machine learning models are very good at finding patterns in a data set. However, the data set that we use for training an model can be quite different from other data sets that we might encounter in the future. Therefore, it is helpful to evaluate how well our model performs on an 'unseen' data set because it can give us a sense of our model performance when used to predict future, unseen data.

To achieve this, the most common approach is to **randomly** split our data set into a training set and a test data. We will only use the training set to train our model. And we will then use the trained model to predict the outcome variable in the test set. The model performance on the test set provides an objective estimate of how well our model performs on future unseen data.

In other words, when we train the model (e.g., linear regression), we should only use the **training set**; when we evaluate the model performance we should only use the **test set**.

The sklearn package allows us to randomly split the data into training and test very easily. Below, we call the train_test_split function, which is part of sklearn package's model_selection utility. The way this function works is that we will provide the predictors (X) and the outcome (Y), a random_state that ensures the random split can be replicated, and the proportion of data that we want to designate as test set (e.g., 0.33 in this case).

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
                          X, y, random_state=42, test_size=.33)

In [ ]:
# examine the shapes of the resulting training set (67% of data) and test set (33% of data)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

## Training Linear Regression

With the training and test sets specified, we are ready to train our linear regression model. We will use the **LinearRegression** object defined in the **sklearn** package.

In [ ]:
from sklearn import linear_model
lr = linear_model.LinearRegression()

To fit a linear regression, simply provide the training set predictors and training set outcome DataFrames as inputs. Then save the trained model as **model** (or any name you like):

In [ ]:
model = lr.fit(X_train, y_train)


We can take peek at the trained model and see what kind of information is generated:

In [ ]:
model.__dict__

In fact, we are only interested in **coef_** (coefficients). Let's match each coefficient with its corresponding column. A positive coefficient means the variable is positively correlated with the outcome variable, holding the values of all other columns constant. Similarly, a negative coefficient means the variable is negatively correlated with the outcome variable.

In [ ]:
pd.DataFrame(model.coef_, X.columns, columns=['Coefficient'])  

We can use the score() function on the model object to assess the performance of our linear regression. The specific measure we generate is called R^2. The larger the R^2, the better the model is. R^2 is between 0 and 1. Notice that we are assessing the model performance on the test data to make sure the estimate is objective.

In [ ]:
print ("R^2 is: \n", model.score(X_test, y_test))


Finally, remember our ultimate goal is to be able to make predictions. This can be achieved easily by calling the predict() function as shown below. Remember, we should predict the data in the test set because those data were not used to train the model and thus more *objective*. 

In [ ]:
predictions = model.predict(X_test)


In order to assess whether the predictions are good, we need to compare the predicted SalePrice with the actual observed SalePrice. Let's put them side by side in a DataFrame:

In [ ]:
pd.DataFrame({'Actual': y_test, 'Predicted': predictions})

We can see that the model actually did a pretty good job predicting the SalePrice.

Most of the times we want to use an objective metric to measure the prediction performance. One such measures, perhaps the most commonly used one, is called the **root mean squared error (RMSE)**. What it does is that it takes the difference between the actual and prediction for each observation (this is the prediction error for each observation), squares it, then takes the mean of the squared difference across all observations. Then it finally takes the square root of that mean squared difference.

Since the RMSE represents some sort of error, **the smaller the better!**

This sounds quite complicated, but to compute this metric we can just call a pre-defined mean squared error in sklearn and take the square root of it:

In [ ]:
from sklearn.metrics import mean_squared_error

# compute the RMSE: this represents the average error for a given observation (~40k)
print ('RMSE is: \n', mean_squared_error(y_test, predictions)**0.5)

#### Make a scatterplot to compare actual vs prediction
We can also visualize the relationship between the predicted price and the actual price through a scatterplot. Ideally the dots should land on the 45 degree line (y=x):

In [ ]:
actual_values = y_test
plt.scatter(predictions, actual_values, alpha=.7,
            color='b') #alpha helps to show overlapping data
plt.xlabel('Predicted Price')
plt.ylabel('Actual Price')
plt.title('Linear Regression Model')
plt.show()

As shown above, we can see that most observations have quite reasonable predictions!